In [ ]:
%pip install google-cloud-aiplatform
%pip install langchain

%pip install faiss-cpu #Facebook AI Similarity Search

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

with open('devops-article.md', 'r') as file:
    text = file.read()

#chunk_size - max number of characters in a chunk
#chunk_overlap - number of characters overlapping between two adjacent chunks
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"], chunk_size=5000, chunk_overlap=350)

docs = text_splitter.create_documents([text])

print (f"{len(docs)} docs ")


In [ ]:
from langchain.embeddings import VertexAIEmbeddings
from langchain.vectorstores import FAISS

#FAISS.from_documents - Return VectorStore initialized from documents and embeddings.
vector_store = FAISS.from_documents(docs, VertexAIEmbeddings())

question = "What are DevOps Transformation Best Practices?"

#similar_docs = vector_store.similarity_search(question, k=2)

similar_docs_with_score = vector_store.similarity_search_with_score(question, k=2)

print(similar_docs_with_score)



In [ ]:
from langchain.llms import VertexAI
from langchain.chains.question_answering import load_qa_chain

llm = VertexAI(
    model_name='text-bison@001',
    temperature=0.0,
    verbose=True,
)

#question = 'What are DevOps Transformation Best Practices?'
question = 'DevOps Maturity Signals related to Development '
#question = 'What are some of the most important DevOps metrics?'

similar_docs = vector_store.similarity_search(question, k=2)

chain = load_qa_chain(llm, chain_type='stuff')

print(chain.run(input_documents=similar_docs, question=question).strip())
